In [1]:
from getpass import getpass
import os

In [2]:
if "SERPERDEV_API_KEY" not in os.environ:
    os.environ["SERPERDEV_API_KEY"] = getpass("Enter SerperDev API key: ")

In [5]:
os.environ["HF_TOKEN"] = getpass("Enter SerperDev API key: ")

In [3]:
from haystack.components.agents import Agent
from haystack.components.generators.chat import HuggingFaceAPIChatGenerator
from haystack.utils import Secret
from haystack.utils.hf import HFGenerationAPIType
from haystack.components.websearch import SerperDevWebSearch
from haystack.dataclasses import ChatMessage
from haystack.tools.component_tool import ComponentTool

/home/eyad/haystack/hastack_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
web_tool = ComponentTool(component=SerperDevWebSearch(), name="web_tool")
agent = Agent(chat_generator=HuggingFaceAPIChatGenerator(api_type=HFGenerationAPIType.SERVERLESS_INFERENCE_API,
                                  api_params={"model": "Qwen/Qwen2.5-7B-Instruct",
                                             "provider": "together"},
                                  token=Secret.from_env_var("HF_TOKEN")), tools=[web_tool])
result = agent.run(messages=[ChatMessage.from_user("Find information about Haystack AI framework")])

print(result["messages"][-1].text)


Haystack is an open-source AI framework developed by deepset AI, designed to help developers build customizable, production-ready applications powered by language models (LLMs), transformer models, and vector search. Here are some key points about Haystack:

1. **Overview and Documentation**: You can find more detailed information on the official Haystack website at [Haystack - deepset AI](https://haystack.deepset.ai/) and its documentation at [Haystack Documentation](https://docs.haystack.deepset.ai/docs/intro).

2. **Purpose**: Haystack is an end-to-end framework that allows you to build applications using LLMs, Transformer models, and vector search. It is particularly useful for document-centric Retrieval-Augmented Generation (RAG) applications and production-ready retrieval pipelines.

3. **Features**: Haystack offers a modular framework that provides full visibility to inspect, debug, and optimize every decision your AI makes. It helps in managing complexity and focuses more on de

In [7]:
from haystack.components.generators.utils import print_streaming_chunk

agent = Agent(
    chat_generator=HuggingFaceAPIChatGenerator(api_type=HFGenerationAPIType.SERVERLESS_INFERENCE_API,
                                  api_params={"model": "Qwen/Qwen2.5-7B-Instruct",
                                             "provider": "together"},
                                  token=Secret.from_env_var("HF_TOKEN")),
    tools=[web_tool], streaming_callback=print_streaming_chunk
)

result = agent.run(messages=[ChatMessage.from_user("Find information about Haystack AI")])
result

[ASSISTANT]
Haystack AI is a platform that provides a suite of tools and services for building and deploying AI models. It is designed to make it easier for developers and data scientists to work with AI, by providing a user-friendly interface and a range of pre-built components and services.

Key features of Haystack AI typically include:

1. **Model Building and Training**: Tools for creating and training machine learning models, including support for various types of models such as text, image, and time-series models.

2. **Data Management**: Services for managing and preprocessing data, including data ingestion, cleaning, and transformation.

3. **Model Deployment**: Options for deploying trained models to production environments, with support for different deployment scenarios and environments.

4. **Monitoring and Maintenance**: Features for monitoring model performance and maintaining models over time, including retraining and updating models as needed.

5. **Integration**: Comp

{'messages': [ChatMessage(_role=<ChatRole.USER: 'user'>, _content=[TextContent(text='Find information about Haystack AI')], _name=None, _meta={}),
  ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[TextContent(text='Haystack AI is a platform that provides a suite of tools and services for building and deploying AI models. It is designed to make it easier for developers and data scientists to work with AI, by providing a user-friendly interface and a range of pre-built components and services.\n\nKey features of Haystack AI typically include:\n\n1. **Model Building and Training**: Tools for creating and training machine learning models, including support for various types of models such as text, image, and time-series models.\n\n2. **Data Management**: Services for managing and preprocessing data, including data ingestion, cleaning, and transformation.\n\n3. **Model Deployment**: Options for deploying trained models to production environments, with support for different de

# Using Agent with a Pipeline as Tool

In [8]:
from haystack.components.builders.answer_builder import AnswerBuilder
from haystack.components.converters.html import HTMLToDocument
from haystack.components.converters.output_adapter import OutputAdapter
from haystack.components.fetchers.link_content import LinkContentFetcher
from haystack.components.websearch.serper_dev import SerperDevWebSearch
from haystack.dataclasses import ChatMessage
from haystack.core.pipeline import Pipeline

In [9]:
search_pipeline = Pipeline()
search_pipeline.add_component("search", SerperDevWebSearch(top_k=10))
search_pipeline.add_component("fetcher", LinkContentFetcher(timeout=3, raise_on_failure=False, retry_attempts=2))
search_pipeline.add_component("converter", HTMLToDocument())
search_pipeline.add_component("output_adapter", OutputAdapter(
    template="""
{%- for doc in docs -%}
  {%- if doc.content -%}
  <search-result url="{{ doc.meta.url }}">
  {{ doc.content|truncate(25000) }}
  </search-result>
  {%- endif -%}
{%- endfor -%}
    """,
    output_type=str
    ),
)

search_pipeline.connect('search.links', 'fetcher.urls')
search_pipeline.connect('fetcher.streams', 'converter.sources')
search_pipeline.connect('converter.documents', 'output_adapter.docs')

🚅 Components
  - search: SerperDevWebSearch
  - fetcher: LinkContentFetcher
  - converter: HTMLToDocument
  - output_adapter: OutputAdapter
🛤️ Connections
  - search.links -> fetcher.urls (list[str])
  - fetcher.streams -> converter.sources (list[ByteStream])
  - converter.documents -> output_adapter.docs (list[Document])

In [10]:
from haystack.core.super_component import SuperComponent
from haystack.tools import ComponentTool
from haystack.components.agents import Agent
from haystack.components.generators.chat import HuggingFaceAPIChatGenerator
from haystack.utils import Secret
from haystack.utils.hf import HFGenerationAPIType

In [11]:
search_component = SuperComponent(
    pipeline=search_pipeline,
    input_mapping={"query": ["search.query"]},
    output_mapping={"output_adapter.output": "search_result"},
)

In [13]:
search_tool = ComponentTool(
    name="search",
    description="Use this tool to search for information on the internet.",
    component=search_component,
    outputs_to_string={"source": "search_result"},
)

In [14]:
agent = Agent(
    chat_generator=HuggingFaceAPIChatGenerator(api_type=HFGenerationAPIType.SERVERLESS_INFERENCE_API,
                                  api_params={"model": "Qwen/Qwen2.5-7B-Instruct",
                                             "provider": "together"},
                                  token=Secret.from_env_var("HF_TOKEN")),
    tools=[search_tool],
    system_prompt="""
    You are a deep research assistant.
    You create comprehensive research reports to answer the user's questions.
    You use the 'search'-tool to answer any questions.
    You perform multiple searches until you have the information you need to answer the question.
    Make sure you research different aspects of the question.
    Use markdown to format your response.
    When you use information from the websearch results, cite your sources using markdown links.
    It is important that you cite accurately.
    """,
    exit_conditions=["text"],
    max_agent_steps=20,
)

In [15]:
query = "What are the latest updates on the Artemis moon mission?"
messages = [ChatMessage.from_user(query)]

agent_output = agent.run(messages=messages)
print(agent_output["messages"][-1].text)

### Latest Updates on the Artemis Moon Mission

#### Overview
NASA's Artemis program aims to return humans to the Moon and establish a sustainable presence there. The Artemis II mission, scheduled for 2024, will send astronauts around the Moon for the first time since 1972. The Artemis III mission, planned for 2027, aims to land the first woman and the first person of color on the Moon.

#### Artemis II Mission
- **Launch Date**: The latest launch date for Artemis II is set for February 2026, with a window of several days each month through April 2026.
- **Crew**: The crew for Artemis II includes NASA astronauts Christina Koch, Reid Wiseman, and Victor Glover, along with Canadian Space Agency astronaut Jeremy Hansen.
- **Orion Spacecraft**: The Orion capsule has arrived at the Vehicle Assembly Building (VAB) at NASA's Kennedy Space Center, where it will be stacked atop the Space Launch System (SLS) rocket.
- **SLS Rocket**: The final piece of the Artemis II SLS rocket, the stage adapte

In [16]:
from IPython.display import Markdown, display

display(Markdown(agent_output["messages"][-1].text))

### Latest Updates on the Artemis Moon Mission

#### Overview
NASA's Artemis program aims to return humans to the Moon and establish a sustainable presence there. The Artemis II mission, scheduled for 2024, will send astronauts around the Moon for the first time since 1972. The Artemis III mission, planned for 2027, aims to land the first woman and the first person of color on the Moon.

#### Artemis II Mission
- **Launch Date**: The latest launch date for Artemis II is set for February 2026, with a window of several days each month through April 2026.
- **Crew**: The crew for Artemis II includes NASA astronauts Christina Koch, Reid Wiseman, and Victor Glover, along with Canadian Space Agency astronaut Jeremy Hansen.
- **Orion Spacecraft**: The Orion capsule has arrived at the Vehicle Assembly Building (VAB) at NASA's Kennedy Space Center, where it will be stacked atop the Space Launch System (SLS) rocket.
- **SLS Rocket**: The final piece of the Artemis II SLS rocket, the stage adapter, has arrived at Kennedy Space Center. This adapter will connect the Orion spacecraft to the rest of the rocket and carry a set of cubesats during the mission.

#### Artemis III Mission
- **Delayed**: The Artemis III mission, originally scheduled for late 2025, has been pushed back to at least mid-2027. This delay is partly due to issues with the Orion spacecraft's heat shield, which experienced more material charring away than expected during the Artemis I test flight.
- **Heat Shield**: NASA is working on addressing the heat shield issues to ensure the crew's safety during the planned mission.

#### Testing and Preparations
- **Plume-Surface Interaction Tests**: NASA is conducting complex plume-surface interaction tests at the Langley Research Center to understand how lunar dust and debris react to lander engine exhaust during landing and takeoffs. These tests are crucial for ensuring the safety of future missions to the Moon and even Mars.
- **Vacuum Chamber Testing**: The tests involve using different propulsion systems, including an ethane plume simulation system and a 3D-printed hybrid rocket motor, to simulate engine exhaust and its effects on lunar regolith.

#### Future Plans
- **Artemis Accords**: NASA is working with international partners to establish a peaceful and transparent environment for space exploration. The Artem